# Hodge Decomposition Python Worksheet

## References

(MAA format)

[1] Sizemore, R. K. (2013) Hodgerank: Applying combinatorial Hodge theory to sports ranking. MA Thesis. Wake Forest University, Winston-Salem, NC. Available at: https://wakespace.lib.wfu.edu/bitstream/handle/10339/38577/Sizemore_wfu_0248M_10444.pdf

[2] Strang, A., Abbott, K. C., Thomas, P. J. (2022) The network HHD: Quantifying cyclic competition in trait-performance models of tournaments. *SIAM Rev.* 64(2): 360-391. DOI: 10.1137/20M1321012

[3] Doyle, P. G., Snell, J. L. (2006) Random walks and electric networks. Available at: https://math.dartmouth.edu/~doyle/docs/walks/walks.pdf

In [2]:
from collections.abc import MutableSequence
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import scipy.sparse as sp

__author__ = "Ina Tang"
output_dir = 'out'
np.set_printoptions(precision=3, suppress=True)

## HodgeRank

A Python implementation of HodgeRank on sports data as described in [1]. 

In [21]:
# Definition found on p. 81. 

class HodgeRanking:
    """
    HodgeRank class for computing rankings based on pairwise comparisons.

    Naming of variables follows the notation in Sizemore.
    
    Attributes
    ----------
    teams : MutableSequence|np.ndarray
        MutableSequence or numpy array of team names in ascending order of their IDs.
    W : np.ndarray
        Weight / adjacency matrix (n x n)
    Y : np.ndarray
        Pairwise comparison matrix (n x n)
    """
    
    def __init__(self, teams, W, Y) -> None:
        self.teams = teams
        self.W = W
        self.Y = Y
    
    @property
    def teams(self) -> MutableSequence|np.ndarray:
        """
        Get the MutableSequence or numpy array of team names.
        
        Returns
        -------
        MutableSequence or np.ndarray of team names in ascending order of their IDs.
        """
        return self._teams
    
    @teams.setter
    def teams(self, value: MutableSequence|np.ndarray) -> None:
        """
        Set the MutableSequence or numpy array of team names.
        
        Parameters
        ----------
        value : MutableSequence or np.ndarray
            List of team names in ascending order of their IDs.
        """
        if not isinstance(value, MutableSequence) and not isinstance(value, np.ndarray):
            raise ValueError("teams must be a MutableSequence or numpy array")
        self._teams = value
    
    @property
    def W(self) -> np.ndarray:
        """
        Get the weight / adjacency matrix.
        
        Returns
        -------
        np.ndarray: Weight / adjacency matrix (n x n)
        """
        return self._W
    
    @W.setter
    def W(self, value: np.ndarray) -> None:
        """
        Set the weight / adjacency matrix.
        
        Parameters
        ----------
        value : np.ndarray
            Weight / adjacency matrix (n x n)
        """
        if not isinstance(value, np.ndarray):
            raise ValueError("W must be a numpy array")
        if value.ndim != 2:
            raise ValueError("W must be a 2D array")
        self._W = value
    
    @property
    def Y(self) -> np.ndarray:
        """
        Get the pairwise comparison matrix.
        
        Returns
        -------
        np.ndarray: Pairwise comparison matrix (n x n)
        """
        return self._Y
    
    @Y.setter
    def Y(self, value: np.ndarray) -> None:
        """
        Set the pairwise comparison matrix.
        
        Parameters
        ----------
        value : np.ndarray
            Pairwise comparison matrix (n x n)
        """
        if not isinstance(value, np.ndarray):
            raise ValueError("Y must be a numpy array")
        if value.ndim != 2:
            raise ValueError("Y must be a 2D array")
        self._Y = value

    @property
    def r(self) -> np.ndarray:
        """
        Get the ranking scores.
        
        Returns
        -------
        np.ndarray: Ranking scores for each team
        """
        return self.rank(self.teams, self.W, self.Y)

    @staticmethod
    def rank(teams, W, Y) -> pd.DataFrame:
        """
        Compute the HodgeRank using the weight matrix W and pairwise comparison matrix Y.
        
        Parameters
        ----------
        W (np.ndarray): Weight / adjacent matrix (n x n)
        Y (np.ndarray): Pairwise comparison matrix (n x n)
        
        Return
        ------
        pd.Dataframe: Ranking scores for each team
        """

        # Laplacian matrix L = D - W, where D is the degree matrix
        L = np.diag(np.sum(W, axis=1)) - W
        
        # Compute rankings
        div_Y = np.sum(Y, axis=1)
        r = np.linalg.pinv(L) @ div_Y
        # np.savetxt('toy-rank.csv', r, fmt='%7.4f', delimiter=',')

        # Format the rankings into a DataFrame and save to CSV
        ranking_df = pd.DataFrame(np.concatenate((np.array(teams).reshape(-1, 1), r.reshape(-1, 1)), axis=1))
        ranking_df.columns = ['team_name', 'score']
        ranking_df = ranking_df.sort_values(by='score', axis=0, ascending=False)
        ranking_df.to_csv(os.path.join(output_dir, 'ranking.csv'), index=False)
        
        return ranking_df
    

# # ----------- Binary Comparisons -----------
# s = np.diag(B @ W.T)   # Diagonal of B * W^T
# D = np.linalg.pinv(L)  # Pseudoinverse of Laplacian
# r = -D @ s             # Compute rankings
# np.savetxt('score_binary.csv', r, fmt='%7.4f', delimiter=',')


## Option 1: Real dataset

Source: [TeamStatistics](https://www.kaggle.com/datasets/eoinamoore/historical-nba-data-and-player-box-scores?select=TeamStatistics.csv) in NBA Dataset - Box Scores & Stats, 1947 - Today (Kaggle)


### Proprocessing

In [22]:
# Load the dataset

# NUM_GAMES = 1000  # Number of games to read

# df = pd.read_csv('TeamStatistics.csv', header=0, index_col=None, nrows=NUM_GAMES)  

# game_dates = pd.to_datetime(df['gameDate'], format='%Y-%m-%d %H:%M:%S')
# print(f"Game date range: {game_dates.min()} to {game_dates.max()}") 

# df.head()  # see year

In [23]:
# df = df[['teamName', 'opponentTeamName', 'win', 'teamScore', 'opponentScore']]
# df = df.rename(columns={
#     'teamName': 'team_name',
#     'opponentTeamName': 'opponent_name',
#     'win': 'team_won',
#     'teamScore': 'team_score',
#     'opponentScore': 'opponent_score'
# })

# df.head()

In [24]:
# # Create a mapping of team names to unique IDs *starting from 0*
# # Note: the matrices will also be zero-indexed
# unique_teams = np.sort(pd.unique(df[['team_name', 'opponent_name']].values.flatten()))
# id_dict: dict[int] = {name: id for id, name in enumerate(unique_teams)}

# num_teams = len(unique_teams)
# print(f"Number of teams: {num_teams}")

# # Map team names to IDs
# df['team_id'] = df['team_name'].map(id_dict)
# df['opponent_id'] = df['opponent_name'].map(id_dict)

# df.head()

### Build matrices

In [25]:
# # 4.2.4 Weight, pairwise comparison, and binary comparison matrices
# # NOT [W^alpha_ij] or [Y^alpha_ij]

# # weight matrix, stores number of pairwise comparisons made, symmetric
# W = np.zeros((num_teams, num_teams), dtype=np.float64) # Use int16 to save memory, modify as needed

# # pairwise comparison matrix, stores rowTeamScore-colTeamScore, antisymmetric
# Y = np.zeros((num_teams, num_teams), dtype=np.float64) 

# # binary comparison matrix, sum of ordinal comparisons (-1, 0, 1) of whether team i beat team j =
# B = np.zeros((num_teams, num_teams), dtype=np.float64)  

# for _, row in df.iterrows():
#     team_id = row['team_id'] 
#     opponent_id = row['opponent_id']  
    
#     # sum up W^alpha_ij s
#     W[team_id, opponent_id] += 1
#     W[opponent_id, team_id] += 1
    
#     # sum up Y^alpha_ij s
#     Y[team_id, opponent_id] += row['team_score'] - row['opponent_score']
#     Y[opponent_id, team_id] += -Y[team_id, opponent_id]

#     if row['team_won']:
#         B[team_id, opponent_id] += 1
#         B[opponent_id, team_id] += -1
#     else:
#         B[team_id, opponent_id] += -1
#         B[opponent_id, team_id] += 1

# # element-wise division to get the average comparison, accounting for zero divisions
# Y = np.divide(Y, W, where=(W != 0))  
# B = np.divide(B, W, where=(W != 0)) 

# print(f"Weight matrix W:\n{W}")
# print(f"Pairwise comparison matrix Y:\n{Y}")
# print(f"Binary comparison matrix B:\n{B}")

## Option 2: Toy examples (4.3.1)

### Example 1: Mostly Consistent Data

In [26]:
teams = np.array(['Mercer', 'Wake Forest', 'Furman', 'Davidson', 'New Mexico', 'George Mason'])

W = np.array([[0,1,1,0,1,1], 
              [1,0,1,0,0,0], 
              [1,1,0,2,0,0], 
              [0,0,2,0,1,0], 
              [1,0,0,1,0,1], 
              [1,0,0,0,1,0]], dtype=np.float64)

Y = np.array([[  0,  -3,   27,     0, -18, -3], 
              [  3,   0,   24,     0,   0,  0], 
              [-27, -24,    0, -27.5,   0,  0], 
              [  0,   0, 27.5,     0,  -5,  0], 
              [ 18,   0,    0,     5,   0,  1],
              [  3,   0,    0,     0,  -1,  0]], dtype=np.float64)

hodge_ranking = HodgeRanking(teams, W, Y)

hodge_ranking.r.head(6)

,team_name,score
4,New Mexico,9.756535947712422
5,George Mason,6.393790849673204
1,Wake Forest,4.334967320261439
0,Mercer,1.0310457516339846
3,Davidson,-2.155228758169933
2,Furman,-19.361111111111114


### Example 2: Inconsistent Data

In [27]:
teams2 = ['Miami', 'NC State', 'Wake Forest', 'Maryland', 'North Carolina', 'Duke']

Y2 = np.array([[  0, -1, -15,   0,   9, 27],
               [  1,  0,  -2,   0,   0,  0],
               [ 15,  2,   0, -26,   0,  0],
               [  0,  0,  26,   0, -10,  0],
               [  9,  0,   0,  10,   0, -5], 
               [-27,  0,   0,   0,   5,  0]], dtype=np.float64)

W2 = np.zeros(Y2.shape, dtype=np.float64)
W2[Y2 != 0] = 1  

hodge_ranking2 = HodgeRanking(teams2, W2, Y2)
hodge_ranking2.r.head(6)

,team_name,score
3,Maryland,7.555555555555556
4,North Carolina,3.5555555555555562
0,Miami,1.8888888888888837
5,Duke,-9.777777777777775
1,NC State,-1.7777777777777803
2,Wake Forest,-1.4444444444444478


In [28]:
# # 4.2.5 Massey's method to verify HodgeRank
# # X is a m × n matrix with X_ki = 1 and X_kj = −1 if team i beats team j in the kth game
# X = np.zeros((NUM_GAMES, num_teams), dtype=np.int8)  # Use int8 to save memory, modify as needed
# for r, row in df.iterrows():  # Set the values in the X matrix
#     team_id = row['team_id']
#     opponent_id = row['opponent_id']
    
#     if row['team_won']:
#         X[r][team_id] = 1
#         X[r][opponent_id] = -1
#     else:
#         X[r][team_id] = -1
#         X[r][opponent_id] = 1

# # vector y stores the margin of victory for each of the m games
# y = np.array(df['team_score'] - df['opponent_score'], dtype=np.int32)  # Use int32 to save memory, modify as needed

# print(f"X matrix:\n{X}")
# print(f"y vector:\n{y}")

# # M & p matrices
# M = np.matmul(X.T, X)  # num_teams x num_teams matrix
# p = np.matmul(X.T, y)  # num_teams x 1 vector

# # Make M non-singular and r = M^(-1)p sum to zero
# M[-1] = 1  # Set the last row to 1 to make it non-singular
# p[-1] = 0  # Set the last element to 0 to make the sum zero

# # Both are final
# print(f"M matrix:\n{M}")
# print(f"p vector:\n{p}")

## Gradient, Divergence, and Curl

Based on Figure 4 in [2]

In [29]:
# Gradient
G = np.array([[-1, 1, 0, 0], 
              [0, -1, 0, 1], 
              [1, 0, 0, -1],
              [0, -1, 1, 0], 
              [0, 0, -1, 1]], dtype=np.float64)

# Divergence
D = G.T

# Curl
C = np.array([[1, 1, 1, 0, 0], 
              [0, -1, 0, 1, 1]], dtype=np.float64)

print(C @ G)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Markov Chains

Page numbers and exercises refer to [3]. 

In [ ]:
# Exercise 1.2.9
# ordered from top to bottom, then left to right

# not canonical
P = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
              [.25, .25, 0, .25, 0, .25, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, .25, 0, .25, 0, .25, 0, .25, 0],
              [0, 0, 0, .25, 0, .25, 0, .25, 0, .25],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=np.float64)

# canonical
P_canonical = np.array([[.25, .25, 0, .25, 0, .25, 0, 0, 0, 0],
                        [0, 0, .25, 0, .25, 0, .25, 0, .25, 0],
                        [0, 0, 0, .25, 0, .25, 0, .25, 0, .25],
                        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=np.float64)

# Raise P to the 10th power
Pn = np.pow(P, 10)
print(f"P matrix:\n{Pn}")

P matrix:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
